In [ ]:
import argparse

## 1.2 add_argument() 方法
ArgumentParser.add_argument(...)

定义单个的命令行参数应当如何解析

### 1.2.1 name or flags
第一个传递给 add_argument() 的参数必须是短选项（`-f`）、长选项（`--foo`）或是一个位置参数

- `parser.add_argument('-f', '--foo')`
- `parser.add_argument('bar')`

### 1.2.2 action
action 命名参数指定了这个命令行参数应当如何处理

In [ ]:
# （1）'store' - 存储参数的值。这是默认的动作 【即 --foo后面的1】
parse = argparse.ArgumentParser()
parse.add_argument('--foo')
print(parse.parse_args('--foo 1'.split()))

In [ ]:
# （2）'store_const' - 存储被 const 命名参数指定的值。 【即 const后面的2】
# 'store_const' 动作通常用在选项中来指定一些标志
parse = argparse.ArgumentParser()
parse.add_argument('--foo', action='store_const', const=2)
print(parse.parse_args(['--foo']))

In [ ]:
# （3）'store_true' and 'store_false' - 这些是 'store_const' 分别用作存储 True 和 False 值的特殊用例。
# 另外，它们的默认值分别为 False 和 True
parse = argparse.ArgumentParser()
parse.add_argument('--foo', action='store_true')
parse.add_argument('--bar', action='store_false')
print(parse.parse_args(['--foo','--bar']))


In [ ]:
# （4）'append' - 存储一个列表，并且将每个参数值追加到列表中。
# 在允许多次使用选项时很有用
parse = argparse.ArgumentParser()
parse.add_argument('--foo', action='append')
print(parse.parse_args(['--foo','1','--foo','2']))

In [ ]:
# （5）'append_const' - 存储一个列表，并将 const 命名参数指定的值追加到列表中。
# （注意 const 命名参数默认为 None。）
# ``'append_const'`` 动作一般在多个参数【--foo --bar】需要在同一列表中存储常数【const=1 const=2】时会有用
parse = argparse.ArgumentParser()
parse.add_argument('--foo', dest='num', action='append_const', const=1)
parse.add_argument('--bar', dest='num', action='append_const', const=2)
print(parse.parse_args(['--foo','--bar']))


In [ ]:
# （6）'count' - 计算一个关键字参数【'-v'】出现的数目或次数。例如，对于一个增长的详情等级来说有用
parser = argparse.ArgumentParser()
parser.add_argument('--verbose', '-v', action='count', default=0)
print(parser.parse_args(['-vv']))
print(parser.parse_args(['-vvv']))

In [ ]:
# （7）'help' - 打印所有当前解析器中的选项和参数的完整帮助信息，然后退出。
# 默认情况下，一个 help 动作会被自动加入解析器。
# 关于输出是如何创建的，参与 ArgumentParser

In [ ]:
#（8）'version' - 期望有一个 version= 命名参数 在 add_argument() 调用中，并打印版本信息并在调用后退出
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--version', action='version', version='%(prog)s 2.0')
print(parser.parse_args(['--version']))

In [ ]:
# （9）'extend' - 这会存储一个列表，并将每个参数值加入到列表中
parser = argparse.ArgumentParser()
parser.add_argument("--foo", action="extend", nargs="+", type=str)
print(parser.parse_args(["--foo", "f1", "--foo", "f2", "f3", "f4"]))

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--foo", action="append", nargs="+", type=str)
print(parser.parse_args(["--foo", "f1", "--foo", "f2", "f3", "f4"]))

传递 Action 子类或实现相同接口的其他对象来指定任意操作。

建议的方法是扩展 Action，覆盖 `__call__` 方法和可选的 `__init__` 方法

In [ ]:
class FooAction(argparse.Action):
    def __init__(self, option_strings, dest, nargs=None, **kwargs):
        if nargs is not None:
            raise ValueError("nargs not allowed")
        super().__init__(option_strings, dest, **kwargs)
    def __call__(self, parser, namespace, values, option_string=None):
        print('%r %r %r' % (namespace, values, option_string))
        setattr(namespace, self.dest, values)

parser = argparse.ArgumentParser()
parser.add_argument('--foo', action=FooAction)
parser.add_argument('bar', action=FooAction)
args = parser.parse_args('1 --foo 2'.split())
print(args)

### 1.2.3 nargs
将不同数目的命令行参数关联到单一动作

如果不提供 nargs 命名参数，则消耗参数的数目将被 action 决定。
通常这意味着单一项目（非列表）消耗单一命令行参数。

In [ ]:
#（1）N （一个整数）。命令行中的 N 个参数会被聚集到一个列表中
parse = argparse.ArgumentParser()
parse.add_argument("--foo",nargs=2)
parse.add_argument("bar",nargs=1)
print(parse.parse_args("c --foo a b".split()))

In [ ]:
parse = argparse.ArgumentParser()
parse.add_argument("--foo")
print(parse.parse_args("--foo a b".split()))
# nargs=1 会产生一个单元素列表。这和默认的元素本身是不同的

In [ ]:
#（2）'?'。如果可能的话，会从命令行中消耗一个参数，并产生一个单一项
parse = argparse.ArgumentParser()
parse.add_argument("--foo",nargs="?", const="c", default="d")
parse.add_argument("bar",nargs="?", default="d")
print(parse.parse_args("xx --foo yy".split()))
print(parse.parse_args("xx --foo".split())) # 选项字符串出现但没有跟随命令行参数，则会使用 const 值。
print(parse.parse_args([])) # 如果当前没有命令行参数，则会使用 default 值。

In [ ]:
# nargs='?' 的一个更普遍用法是 使用可选的输入或输出文件:
import sys
parser = argparse.ArgumentParser()
parser.add_argument('infile', nargs='?', type=argparse.FileType('r'),
                    default=sys.stdin)
parser.add_argument('outfile', nargs='?', type=argparse.FileType('w'),
                    default=sys.stdout)
print(parser.parse_args(['input.txt', 'output.txt']))
print(parser.parse_args([]))

In [ ]:
# （3）'*'。所有当前命令行参数被聚集到一个列表中。
# 注意通过 nargs='*' 来实现多个位置参数通常没有意义，但是多个选项是可能的
parser = argparse.ArgumentParser()
parser.add_argument('--foo', nargs='*')
parser.add_argument('--bar', nargs='*')
parser.add_argument('baz', nargs='*')
print(parser.parse_args('a b --foo x y --bar 1 2'.split()))

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('bar', nargs='*')
parser.add_argument('baz', nargs='*')
print(parser.parse_args('a b 1 2'.split()))

In [ ]:
# （4）'+'。和 '*' 类似，所有当前命令行参数被聚集到一个列表中。
# 另外，当前没有至少一个命令行参数时会产生一个错误信息。例如:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('foo', nargs='+')
print(parser.parse_args(['a', 'b']))
print(parser.parse_args([]))

In [ ]:
# （5）argarse.REMAINDER。所有剩余的命令行参数被聚集到一个列表中。
# 这通常在从一个命令行功能传递参数到另一个命令行功能中时有用:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--foo')
parser.add_argument('command')
parser.add_argument('args', nargs=argparse.REMAINDER)
print(parser.parse_args('--foo B cmd --arg1 XX ZZ'.split()))

### 1.2.4 const

保存不从命令行中读取但被各种 ArgumentParser 动作需求的常数值。

最常用的两例为：

- 当 add_argument() 通过 action='store_const' 或 action='append_const 调用时。这些动作将 const 值添加到 parse_args() 返回的对象的属性中。在 action 的描述中查看案例。

- 当 add_argument() 通过选项（例如 -f 或 --foo）调用并且 nargs='?' 时。这会创建一个可以跟随零个或一个命令行参数的选项。当解析命令行时，如果选项后没有参数，则将用 const 代替。在 nargs 描述中查看案例。

对 'store_const' 和 'append_const' 动作， const 命名参数必须给出。对其他动作，默认为 None。

### 1.2.4 default

默认值为 None，指定了在命令行参数未出现时应当使用的值。


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', default=42)
print(parser.parse_args(['--foo', '2']))
print(parser.parse_args([]))

In [ ]:
# 在将属性设置在 Namespace 的返回值之前，解析器使用提供的 type 转换参数。
# 否则解析器使用原值:
parser = argparse.ArgumentParser()
parser.add_argument('--length', default='10', type=int)
parser.add_argument('--width', default=10.5, type=int)
parser.add_argument('--height', default='20')
print(parser.parse_args([]))

In [ ]:
# 对于 nargs 等于 ? 或 * 的位置参数， default 值在没有命令行参数出现时使用。
parser = argparse.ArgumentParser()
parser.add_argument('foo', nargs='?', default=42)
print(parser.parse_args(['a']))
print(parser.parse_args([]))

In [ ]:
# 提供 default=argparse.SUPPRESS 导致命令行参数未出现时没有属性被添加:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', default=argparse.SUPPRESS)
print(parser.parse_args([]))
print(parser.parse_args(['--foo', '1']))

### 1.2.5 type

默认情况下，ArgumentParser 对象将命令行参数当作简单字符串读入。

type 允许任何的类型检查和类型转换

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('foo', type=int)
parser.add_argument('bar', type=open)
print(parser.parse_args('2 input.txt'.split()))

当 type 参数被应用到默认参数时，请参考 default 参数的部分。

为方便使用不同类型的文件，argparse 模块提供了 FileType 工厂类，
该类接受 `mode=`, `bufsize=`, `encoding=` 和 `errors=` 等 open() 函数参数。 

例如，FileType('w') 可被用来创建一个可写文件:

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('bar', type=argparse.FileType('w'))
print(parser.parse_args(['out.txt']))

`type=` 可接受任意可调用对象，该对象应传入单个字符串参数并返回转换后的值: 【类型检查】

In [ ]:
import math
def perfect_square(string):
    value = int(string)
    sqrt = math.sqrt(value)
    if sqrt != int(sqrt):
        msg = "%r is not a perfect square" % string
        raise argparse.ArgumentTypeError(msg)
    return value

parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('foo', type=perfect_square)
print(parser.parse_args(['9']))
# print(parser.parse_args(['7']))
# usage: PROG [-h] foo
# PROG: error: argument foo: '7' is not a perfect square

choices 关键词参数可能会使类型检查者更方便的检查一个范围的值。

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('foo', type=int, choices=range(5, 10))
print(parser.parse_args(['7'])) # choices 容器包含的内容会在执行任意 type 转换之后被检查
# print(parser.parse_args(['11']))
# usage: PROG [-h] {5,6,7,8,9}
# PROG: error: argument foo: invalid choice: 11 (choose from 5, 6, 7, 8, 9)

### 1.2.6 choices

判断命令行参数是不是在 choices 指定的一个容器对象中，如果不在将显示错误消息。

choices 容器包含的内容会在执行任意 type 转换之后被检查。

任何容器都可作为 choices 值传入，因此 list 对象，set 对象以及自定义容器都是受支持的。


### 1.2.7 required

要让一个选项成为 必需的，则可以将 True 作为 required= 关键字参数传给 add_argument():

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', required=True)
print(parser.parse_args(['--foo', 'BAR']))
print(parser.parse_args([]))

### 1.2.8 help 

值是一个包含参数简短描述的字符串。 

当用户请求帮助时（一般是通过在命令行中使用 -h 或 --help 的方式），这些 help 描述将随每个参数一同显示:

In [ ]:
parser = argparse.ArgumentParser(prog='frobble')
parser.add_argument('--foo', 
                    action='store_true',
                    help='foo the bars before frobbling')
parser.add_argument('bar', 
                    nargs='+',
                    help='one of the bars to be frobbled')

print(parser.parse_args(['-h']))

help 字符串可包括各种【格式描述符】，以避免重复使用程序名称或参数 default 等文本。 

有效的描述符包括程序名称 %(prog)s 和传给 add_argument() 的大部分关键字参数，

例如 %(default)s, %(type)s 等等:

In [ ]:
parser = argparse.ArgumentParser(prog='frobble')
parser.add_argument('bar', nargs='?', type=int, default=42,
                    help='the bar to %(prog)s (default: %(default)s)')
parser.print_help()

由于帮助字符串支持 `%-formatting`，如果你希望在帮助字符串中显示 % 字面值，你必须将其转义为 %%。

argparse 支持静默特定选项的帮助，具体做法是将 help 的值设为 `argparse.SUPPRESS`:

In [ ]:
parser = argparse.ArgumentParser(prog='frobble')
parser.add_argument('--foo', help=argparse.SUPPRESS)
parser.print_help()

### 1.2.9 metavar

默认情况下，ArgumentParser 对象使用 dest 值作为每个对象的 "name"。 

默认情况下，

- 对于位置参数动作，dest 值将被直接使用     【`bar` --> bar】
- 对于可选参数动作，dest 值将被转为大写形式 【`--foo` --> FOO】

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo')
parser.add_argument('bar')
print(parser.parse_args('X --foo Y'.split()))
parser.print_help()

可以使用 metavar 来指定一个替代名称:

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', metavar='YYY')
parser.add_argument('bar', metavar='XXX')
print(parser.parse_args('X --foo Y'.split()))
parser.print_help()

请注意 metavar 仅改变 显示的 名称 - parse_args() 对象的属性名称仍然会由 dest 值确定。

不同的 nargs 值可能导致 metavar 被多次使用。 

提供一个元组给 metavar 即为每个参数指定不同的显示信息:  【metavar=('bar', 'baz')】

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-x', nargs=2)
parser.add_argument('--foo', nargs=2, metavar=('bar', 'baz'))
parser.print_help()

### 1.2.11 dest

parse_args() 所返回对象的一个属性的名称由 add_argument() 的 dest 关键字参数确定。

【`args = parser.parse_args()` 。args 调用的属性名由 dest 关键字参数确定】

对于【位置参数】动作，dest 通常会作为 add_argument() 的第一个参数提供  【使用默认的名称】

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('bar')
args = parser.parse_args(['XXX'])
print(args)
print(args.bar)

对于【可选参数】动作，dest 的值通常取自选项字符串。  【使用默认的名称】

ArgumentParser 会通过接受第一个长选项字符串并去掉开头的 `--` 字符串来生成 dest 的值。【即`foo-bar`】

如果没有提供长选项字符串，则 dest 将通过接受第一个短选项字符串并去掉开头的 `-` 字符来获得。【即`x`】

任何内部的 `-` 字符都将被转换为 `_` 字符以确保字符串是有效的属性名称 【Namespace(foo_bar='1', x='2')】

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-f', '--foo-bar', '--foo')
parser.add_argument('-x', '-y')
print(parser.parse_args('-f 1 -x 2'.split()))
print(parser.parse_args('--foo 1 -y 2'.split()))

使用 dest 自定义属性名称  【使用自定义的名称】

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', dest='bar')
print(parser.parse_args('--foo XXX'.split()))

### 1.2.12 Action 类

`class argparse.Action(option_strings, dest, nargs=None, const=None, 
                      default=None, type=None, choices=None, required=False, 
                      help=None, metavar=None)`

返回可调用对象的可调用对象，作为 action 形参传给 add_argument()，返回的可调用对象可处理来自命令行的参数。

Action 类必须接受两个位置参数【option_strings, dest】以及传给 `ArgumentParser.add_argument()` 的任何关键字参数，除了 action 本身。

Action 的实例（或作为or return value of any callable to the action 形参的任何可调用对象的返回值）应当定义
 "dest", "option_strings", "default", "type", "required", "help" 等属性。 
 确保这些属性被定义的最容易方式是 调用 `Action.__init__`。

Action 的实例应当为可调用对象，因此所有子类都必须重载 `__call__` 方法，该方法应当接受四个形参:

- parser - 包含此动作的 ArgumentParser 对象。

- namespace - 将由 parse_args() 返回的 Namespace 对象。 大多数动作会使用 setattr() 为此对象添加属性。

- values - 已关联的命令行参数，并提供相应的类型转换。 类型转换由 add_argument() 的 type 关键字参数来指定。

- option_string - 被用来发起调用此动作的选项字符串。 option_string 参数是可选的，且此参数在动作关联到位置参数时将被略去。

`__call__` 方法可以执行任意动作，但通常将基于 dest 和 values 来设置 namespace 的属性

In [ ]:
class FooAction(argparse.Action):
    def __init__(self, option_strings, dest, nargs=None, **kwargs):
        if nargs is not None:
            raise ValueError("nargs not allowed")
        super().__init__(option_strings, dest, **kwargs)
    def __call__(self, parser, namespace, values, option_string=None):
        print('%r %r %r' % (namespace, values, option_string))
        setattr(namespace, self.dest, values)

parser = argparse.ArgumentParser()
parser.add_argument('--foo', action=FooAction)
parser.add_argument('bar', action=FooAction)
args = parser.parse_args('1 --foo 2'.split())
print(args)

## 1.3 parse_args() 方法

ArgumentParser.parse_args(args=None, namespace=None)

将参数字符串转换为对象并将其设为命名空间的属性。 返回带有成员的命名空间。

### 1.3.1 选项值语法

指定选项值的方式

In [ ]:
parse = argparse.ArgumentParser()
parse.add_argument("-f","--foo", default="x")
parse.add_argument("-b","--bar", default="y")

print(parse.parse_args(["--foo","XX","--bar","YY"]))
print(parse.parse_args(["--foo","XX"]))
print(parse.parse_args(["--foo=XX"]))  # 对于长选项（名称长度超过一个字符的选项）
print(parse.parse_args(["-bYY"])) # 对于短选项（长度只有一个字符的选项）

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-x', action='store_true')
parser.add_argument('-y', action='store_true')
parser.add_argument('-z')
print(parser.parse_args(['-xyzZ'])) # 如果仅有最后一个选项（或没有任何选项）需要值的话,有些短选项可以使用单个 - 前缀来进行合并

### 1.3.2 无效的参数
parse_args() 会检测多种错误，包括有歧义的选项、无效的类型、无效的选项、错误的位置参数个数等等。 
当遇到这种错误时，它将退出并打印出错误文本同时附带用法消息

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--foo', type=int)
parser.add_argument('bar', nargs='?')

# invalid type
# PROG: error: argument --foo: invalid int value: 'spam'
print(parser.parse_args(['--foo', 'spam']))

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--foo', type=int)
parser.add_argument('bar', nargs='?')
# invalid option
print(parser.parse_args(['--bar']))

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--foo', type=int)
parser.add_argument('bar', nargs='?')
# wrong number of arguments
print(parser.parse_args(['spam', 'badger']))

### 1.3.3 包含 - 的参数

位置参数只有在它们看起来像负数，并且解析器中没有任何选项看起来像负数时才能以 `-` 打头

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-x')
parser.add_argument('foo', nargs='?')

# no negative number options, so -1 is a positional argument
print(parser.parse_args(['-x', '-1']))

# no negative number options, so -1 and -5 are positional arguments
print(parser.parse_args(['-x', '-1', '-5']))

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-1', dest='one')
parser.add_argument('foo', nargs='?')

# negative number options present, so -1 is an option
print(parser.parse_args(['-1', 'X']))

# negative number options present, so -2 is an option
print(parser.parse_args(['-2']))

# negative number options present, so both -1s are options
print(parser.parse_args(['-1', '-1']))

如果你有必须以 `-` 打头的位置参数并且看起来不像负数，
你可以插入伪参数 `--` 以告诉 parse_args() 在那之后的内容是一个位置参数:

In [ ]:
print(parser.parse_args(['--', '-f']))

### 1.3.4 参数缩写（前缀匹配）

在默认情况下 如果缩写无歧义，那么长选项可以缩写为前缀

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-bacon')
parser.add_argument('-badger')
print(parser.parse_args('-bac MMM'.split()))
print(parser.parse_args('-bad WOOD'.split()))
print(parser.parse_args('-ba BA'.split()))

In [ ]:
# 通过将 allow_abbrev 设为 False 来禁用缩写，来避免错误
parser = argparse.ArgumentParser(prog='PROG', allow_abbrev=False)
parser.add_argument('-bacon')
parser.add_argument('-badger')
print(parser.parse_args('-bac MMM'.split()))

### 1.3.5 在 sys.argv 以外

在 sys.argv 以外【除命令行外】用 ArgumentParser 解析参数也是有用的。 

这可以通过将一个字符串列表传给 parse_args() 来实现。 它适用于在交互提示符下进行检测

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument(
    'integers', metavar='int', type=int, choices=range(10),
    nargs='+', help='an integer in the range 0..9')
parser.add_argument(
    '--sum', dest='accumulate', action='store_const', const=sum,
    default=max, help='sum the integers (default: find the max)')
print(parser.parse_args(['1', '2', '3', '4']))
print(parser.parse_args(['1', '2', '3', '4', '--sum']))

### 1.3.6 命名空间对象

`class argparse.Namespace`

由 parse_args() 默认使用的简单类，可创建一个存放属性的对象并将其返回。

这个类被有意做得很简单，只是一个具有可读字符串表示形式的 object。 

如果你更喜欢类似字典的属性视图，你可以使用标准 Python 中惯常的 vars():

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo')
args = parser.parse_args(['--foo', 'BAR'])
print(vars(args))


指定一个命名空间：

In [ ]:
class C:
    pass

c = C()
parser = argparse.ArgumentParser()
parser.add_argument('--foo')
parser.parse_args(args=['--foo', 'BAR'], namespace=c)
print(c.foo)


## 1.4 其它实用工具

TODO

[https://docs.python.org/zh-cn/3.8/library/argparse.html#other-utilities](https://docs.python.org/zh-cn/3.8/library/argparse.html#other-utilities)

